In [12]:
import sqlite3
import pandas as pd
import mariadb
import sys
import os
from dotenv import load_dotenv
import re

In [13]:
load_dotenv()

True

In [37]:
#database location
db_location = 'scripts/crawlers/usedcarsni/database.db'

def read_database(db_path):
    conn = sqlite3.connect(db_path)
    query = "SELECT * FROM details_cars_data"
    df = pd.read_sql_query(query, conn)
    conn.close()

    return df

In [94]:
df = read_database(db_location)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3161 entries, 0 to 3160
Data columns (total 24 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   title                      3161 non-null   object
 1   price                      3061 non-null   object
 2   year                       3161 non-null   object
 3   milage                     3161 non-null   object
 4   transmission               3161 non-null   object
 5   fuel_type                  3161 non-null   object
 6   body_style                 3161 non-null   object
 7   engine_size                3161 non-null   object
 8   doors                      3161 non-null   object
 9   location                   3161 non-null   object
 10  link                       3140 non-null   object
 11  phone                      3161 non-null   object
 12  name                       3161 non-null   object
 13  address                    3161 non-null   object
 14  mot_expi

In [95]:
def extract_year(string):
    pattern = r'\b(19|20)\d{2}\b'  # Matches a four-digit number starting with 19 or 20
    match = re.search(pattern, string)
    if match:
        return match.group()
    else:
        return None

def check_car_make(string):
    car_makes = ['Acura', 'Alfa Romeo', 'Aston Martin', 'Audi', 'Bentley', 'BMW', 'Bugatti', 'Buick', 'Cadillac', 'Chevrolet', 'Chrysler', 'Citroen', 'Dodge', 'Ferrari', 'Fiat', 'Ford', 'Genesis', 'GMC', 'Honda', 'Hummer', 'Hyundai', 'Infiniti', 'Jaguar', 'Jeep', 'Kia', 'Lamborghini', 'Land Rover', 'Lexus', 'Lincoln', 'Lotus', 'Maserati', 'Mazda', 'McLaren', 'Mercedes-Benz', 'Mini', 'Mitsubishi', 'Nissan', 'Pagani', 'Peugeot', 'Porsche', 'Ram', 'Renault', 'Rolls-Royce', 'Saab', ' Seat', 'Subaru', 'Suzuki', 'Tesla', 'Toyota', 'Volkswagen', 'Volvo', 'Vauxhall']
    
    for make in car_makes:
        if make.lower() in string.lower():
            return make
    
    return None

def convert_price(price_str):
    # Check if the price string is missing or invalid
    if pd.isna(price_str) or not re.match(r'£\d{1,3}(,\d{3})*(\.\d{1,2})?$', price_str):
        return None
    # Convert the price string to an integer value
    price_int = int(price_str.replace('£', '').replace(',', ''))
    return price_int




In [96]:
#extract year from title
df['year'] = df['title'].apply(extract_year)
#extract make from title
df['make'] = df['title'].apply(check_car_make)
#convert price or drop record if none or not match using re
df['price'] = df['price'].apply(convert_price)

#cleaning milage column
df['milage'] = df['milage'].str.replace('Miles','').astype(int)

#cleaning engine_size column
def clean_egine_size(engine_size):
    try:
        return engine_size.replace('L','')
    except (ValueError, AttributeError):
        return None
df['engine_size'] = df['engine_size'].apply(clean_egine_size)

#clean doors column

def clean_doors(doors):
    try:
        return doors.replace('door','')
    except (ValueError, AttributeError):
        return None
df['doors'] = df['doors'].apply(clean_doors)

#clean location column
def clean_location(location):
    try:
        return location.capitalize()
    except (ValueError, AttributeError):
        return None

df['location'] = df['location'].apply(clean_location)

#clean all archived listings
df = df[~df['title'].str.contains('Archived')]

#drop N/A
df = df.dropna()

In [98]:
df.to_csv('test.csv')

In [7]:
# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user=os.getenv('MARIADB_USER'),
        password=os.getenv('MARIADB_PASSWORD'),
        host=os.getenv('MARIADB_HOST'),
        port=3307,
        database=os.getenv('MARIADB_DATABASE')

    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

Error connecting to MariaDB Platform: Can't connect to server on 'localhost' (10061)


AttributeError: 'tuple' object has no attribute 'tb_frame'